In [186]:
# найти файл по вкладке и вывести размер вкладки
def findfile(sheetname):
    row = 0
    col = 0
    for name, df_l in df_dicts:
        try:
            col = df_l[sheetname].shape[1]
            df_l[sheetname]
            print(name)
        except:
            continue
        try:
            row += df_l[sheetname].shape[0] 
            if (abs(col - col)<1):
                   continue
            else:
                col += abs(df_l[sheetname].shape[1] - col)
        except:
            continue
    print('rows = %d, col = %d' %(row, col))

In [187]:
import pandas as pd
import numpy as np
import warnings
import re
from datetime import datetime
from collections import OrderedDict
warnings.filterwarnings('default')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

In [188]:
# подгрузка всех файлов
import os
dr = 'exp'
file_names = os.listdir(dr)
file_names.remove('prepared')
# файлы в список
df_dicts = {}
for name in file_names:
    df_dicts[name]= pd.read_excel(dr+'\\'+name, None)
#     print(name)

In [189]:
# список всех таблиц из всех файлов
sheetname_list = []
for name, dict_ in zip(df_dicts, df_dicts.values()):    
    tmp = dict_.keys()
    tmp = list(tmp)
    tmp = tmp[2:-3]
    sheetname_list.append(tmp)
len(sheetname_list)

28

In [190]:
# sheetname_list['февраль 2018-испытания.xlsx']

In [191]:
# dict_sheet_count - словарь всех вкладок(листов) из таблиц без дубликатов и их частоты
# uniq_sheetname - множество всех вкладок(листов) из таблиц без дубликатов (не используется)

dict_sheet_count = {}
uniq_sheetname = set()
for st in sheetname_list:
    for ls in st:
        uniq_sheetname.add(ls)


uniq_sheetname = list(uniq_sheetname)
uniq_sheetname.sort()
print(len(uniq_sheetname))

# добавление частот в словарь
for st in sheetname_list:
    for ls in st:
        counts = dict_sheet_count.get(ls,0)
        dict_sheet_count[ls] = counts + 1
        
        
# df - для вывода словаря в excel            
df = pd.DataFrame.from_dict(data = dict_sheet_count, orient='index')
# df.to_excel('dict_sheet_count.xlsx')

103


In [192]:
ls_mb_convert_ys = [
    'Предел текучести (1141)',
    'Предел текучести (1027)',
    'Предел текучести, Н/мм2 или МПа',
    'Предел текучести , кгс/мм2',
    'Предел текучести, кгс/мм2',
]

ls_mb_convert_tr = [
    'Врем. сопротивление,  Н/мм2 или МПа',
    'Врем. сопротивление (1140)',
    'Врем. сопротивление (1042)',
    'Врем. сопротивление , кгс/мм2',
    'Врем. сопротивление, кгс/мм2'
]

ls_mb_convert = ls_mb_convert_ys+ls_mb_convert_tr

In [193]:
def change_col_name(df):
    for col in ls_mb_convert_tr:
        if col in df:
            df['Врем. сопротивление'] = df[col]
            del df[col]
    for col in ls_mb_convert_ys:
        if col in df:
            df['Предел текучести'] = df[col]
            del df[col]
    return df

In [194]:
ls_sheets = [
    u'Растяжение, кгс мм2.',
    u'Растяжение, н мм2.',
]

In [195]:
scretching = pd.DataFrame()
for filename in file_names:
    tmp = df_dicts[filename][u'Растяжение, кгс мм2.']
    tmp = change_col_name(tmp)
    scretching= pd.concat([tmp, scretching])
# scretching[~pd.isnull(scretching['Врем. сопротивление'])].shape, scretching.shape

In [196]:
scretching2  = pd.DataFrame()
for filename in file_names:
    tmp = df_dicts[filename][u'Растяжение, н мм2.']
    tmp = change_col_name(tmp)
    scretching2= pd.concat([tmp, scretching2])

In [197]:
scretching.shape, scretching2.shape

((21487, 33), (36069, 43))

In [198]:
scretching['Врем. сопротивление'].describe()

count    21196.000000
mean        72.022825
std         38.484628
min         35.500000
25%         56.500000
50%         69.500000
75%         82.500000
max        995.500000
Name: Врем. сопротивление, dtype: float64

In [199]:
scr_df = scretching[[
    '№ плавки',
    '№ партии',
    'Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести' ]]

In [200]:
scr_df2 = scretching2[[
    '№ плавки',
    '№ партии',
    'Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести' ]]


In [201]:
scr_df.shape, scr_df2.shape

((21487, 7), (36069, 7))

In [157]:
# scr_df2['Тип предела текучести (1186)'].value_counts()
# describe()

In [158]:
# scr_df= scr_df[scr_df['Тип предела текучести (1186)']==0.2]
# .describe()

In [159]:
# scr_df2['Тип предела текучести (1186)'] = scr_df2['Тип предела текучести (1186)'].apply(lambda x: str(x))
# scr_df2= scr_df2[scr_df2['Тип предела текучести (1186)']=='0.2']
# .describe()

In [202]:
scr_df.shape, scr_df2.shape

((21487, 7), (36069, 7))

In [203]:
scr_df['Врем. сопротивление'] = scr_df['Врем. сопротивление'].apply(lambda x: x/9.8 if x>200 else x)
scr_df['Предел текучести'] = scr_df['Предел текучести'].apply(lambda x: x/9.8 if x>200 else x)

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [204]:
scr_df2['Врем. сопротивление'] = scr_df2['Врем. сопротивление'].apply(lambda x: x/9.8 if x>200 else x)
scr_df2['Предел текучести'] = scr_df2['Предел текучести'].apply(lambda x: x/9.8 if x>200 else x)

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [205]:
scr_df['№ партии'] = scr_df['№ партии'].astype(str)
scr_df2['№ партии'] = scr_df2['№ партии'].astype(str)

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [206]:
scr_df = pd.concat([scr_df, scr_df2])

In [207]:
scr_df.drop_duplicates(inplace=True)

In [208]:
scr_df.shape

(56127, 7)

In [ ]:
# scr_df[scr_df.duplicated(['№ плавки', '№ партии'], keep=False)].sort_values('№ партии')

In [210]:
# scr_df.to_excel('table_stretching_13april.xlsx')

In [211]:
scr_df['№ партии']=scr_df['№ партии'].apply(lambda x: str(x))

In [212]:
scr_df.shape

(56127, 7)

In [213]:
scr_df.drop_duplicates(inplace = True)
scr_df.shape

(56127, 7)

In [214]:
scr_df['Дата форм. цехом'] = scr_df['Дата форм. цехом'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y"))
# dt = datetime.strptime("01.02.2016", "%d.%m.%Y")
# dt

In [215]:
scr_df.reset_index(inplace=True,drop=True)

In [216]:
undupl = scr_df[~scr_df[['№ плавки', '№ партии']].duplicated(keep = False)]

In [217]:
dupl_need_to_mean = scr_df[scr_df[['№ плавки', '№ партии']].duplicated(keep = False)]
# sort_values(['№ партии', 'Дата форм. цехом'])

In [218]:
dupl_need_to_mean.shape, undupl.shape, dupl_need_to_mean.shape[0]+undupl.shape[0]

((44626, 7), (11501, 7), 56127)

In [219]:
# d = {1:['11', '12'], 2:['21', '22']}
# ttt = pd.DataFrame(d)
# ttt
# ttt[1].iloc[0]

In [220]:
def mean_func(x):
    if min(x['Дата форм. цехом'])!=max(x['Дата форм. цехом']):
        x = x[x['Дата форм. цехом']==max(x['Дата форм. цехом'])]
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
#         x['Марка стали'] = x['Марка стали'].iloc[0]
    else:
        x['Врем. сопротивление'] = x['Врем. сопротивление'].mean()
        x['Предел текучести'] = x['Предел текучести'].mean()
#         x['Марка стали'] = x['Марка стали'].iloc[0]
    return x.mean()

In [221]:
y = dupl_need_to_mean.groupby(['№ плавки', '№ партии'])['Дата форм. цехом',
    'Марка стали',
    'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести'].apply(mean_func)

In [222]:
y.reset_index(inplace=True)
# y.sort_values(['№ партии'])

In [223]:
# dupl_need_to_mean.sort_values(['№ партии', 'Дата форм. цехом'])

In [224]:
# y[y[['№ плавки', '№ партии']].duplicated(keep = False)].sort_values(['№ партии'])

In [225]:
tmp = undupl[['№ плавки', '№ партии', 'Тип предела текучести (1186)',
    'Врем. сопротивление',
    'Предел текучести']]
y = pd.concat([y, tmp])

In [228]:
y.shape

(30332, 5)

In [227]:
y.to_excel('table_stretching_17april.xlsx')

In [182]:
y['Тип предела текучести (1186)'].describe()

count    29818.000000
mean         0.341440
std          0.151549
min          0.200000
25%          0.200000
50%          0.200000
75%          0.500000
max          0.600000
Name: Тип предела текучести (1186), dtype: float64

In [ ]:
# y